# Imports

In [5]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressionModel, RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
import pyspark.sql
import pandas

from sklearn.ensemble import RandomForestRegressor

# Read in intermediate data files

In [2]:
training_main = spark.read.option("delimiter", "\t").csv("newtrain/*.csv", header=True)

# (Pandas implementation)

In [19]:
train_pd = training_main.toPandas()

In [20]:
train_pd = train_pd.drop("Click", 1).drop("Impression", 1)

In [21]:
train_pd = train_pd.drop("QueryTokens", 1).drop("TitleTokens", 1).drop("DescriptionTokens", 1).drop("KeywordTokens", 1)

In [22]:
labels = train_pd["MainCTRNormd"]

In [23]:
train_pd = train_pd.drop("MainCTRNormd", 1)

In [39]:
train_pd = train_pd.drop(train_pd.columns[0:12], 1)

In [40]:
train_pd.columns

Index([u'Age', u'DisplayURL_CTR_normd', u'AdID_CTR_normd',
       u'AdvertiserID_CTR_normd', u'QueryID_CTR_normd', u'KeywordID_CTR_normd',
       u'TitleID_CTR_normd', u'DescriptionID_CTR_normd', u'UserID_CTR_normd'],
      dtype='object')

In [50]:
rf_model = RandomForestRegressor().fit(train_pd.iloc[:150000], labels.iloc[:150000])

In [51]:
x_trans = rf_model.predict(train_pd)

In [55]:
train_pd["predictions"] = x_trans
train_pd["labels"] = labels

In [56]:
train_pd.to_csv("train_and_test_predictions.csv")

In [42]:
score = rf_model.score(train_pd.iloc[150000:], labels.iloc[150000:])

In [43]:
score

0.77672861922041658

In [37]:
rf_model.feature_importances_

array([  6.02892845e-03,   6.87168103e-03,   5.75313639e-03,
         7.03217795e-04,   3.80814193e-03,   1.69092926e-02,
         7.80835957e-03,   7.34577371e-03,   5.58060681e-03,
         2.21996378e-03,   1.34142102e-02,   2.76647238e-04,
         7.15129012e-04,   4.99257994e-03,   2.25387832e-01,
         6.46392436e-03,   6.41029153e-01,   1.48879382e-02,
         2.40599298e-02,   4.52330964e-03,   1.22024437e-03])

In [9]:
training_main = training_main.limit(25000).drop("QueryTokens").drop("KeywordTokens").drop("TitleTokens").drop("DescriptionTokens")

In [11]:
training_main.count()

25000

In [12]:
string_cols = ["DisplayURL", "AdID", "AdvertiserID", "QueryID", "KeywordID",
                                "TitleID", "DescriptionID", "UserID", "Gender"]

onehot_cols = []
ctr_cols = []

# Pass columns through StringIndexer and OneHotEncoder

In [13]:
for column in string_cols:
    # set indexer columns
    str_index = StringIndexer().setInputCol(column).setOutputCol(column + "_index").fit(training_main)
    
    training_main = str_index.transform(training_main)

    # set one hot encoder columns
    onehotencoder = OneHotEncoder().setInputCol(column + "_index").setOutputCol(column + "_onehot")
    # transform
    training_main = onehotencoder.transform(training_main)

    # add the new column to our list
    onehot_cols.append(column + "_onehot")
    if(column != "Gender"):
        ctr_cols.append(column + "_CTR_normd")

In [14]:
assembler = VectorAssembler().setInputCols(["UserID_onehot", "Gender_onehot", "TitleID_onehot", "QueryID_onehot", "DescriptionID_onehot", "KeywordID_onehot", "AdvertiserID_onehot", "AdID_onehot"]).setOutputCol("features")

training_main = assembler.transform(training_main)

In [15]:
training_main = training_main.withColumn("label", training_main["MainCTRNormd"].cast("Double"))

# Convert to RDD

In [16]:
trn = training_main.select("features", "label")
train_rdd = trn.rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))

In [17]:
trn.count()

25000

# Build model

In [22]:
#lr_model = RandomForest.trainRegressor(train_rdd, {}, 3)
lr_model = RandomForest.trainRegressor(train_rdd, {}, 2)

KeyboardInterrupt: 

In [9]:
# paramGrid = ParamGridBuilder().build()

# evaluator = RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")

# rf = RandomForestRegressor().setLabelCol("label").setFeaturesCol("features").setPredictionCol("prediction")

# cv = CrossValidator().setEstimator(rf).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(5)
# cvModel = cv.fit(training_main)

# prediction_table = cvModel.transform(training_main)

In [ ]:
training_main.show()